In [34]:
import pandas as pd
# Primer paso, importado de los datos

datos = pd.read_csv("Datos/datos.csv")

In [35]:
datos.head(2)

,nom,edad,sex,rel_ctxo_rel_mala,rel_ctxo_trauma,rel_ctxo_buena,ed_perm,ed_norm,ed_estr,resil_ba,...,max_min,conc_arb,pseu_res,deb,raz_emo,inhib,asert,agres,impuls,grupo
0,Elena,20,0,1,0,0,0,1,0,0,...,1,1,0,1,1,0,0,1,1,1
1,Marta,19,0,0,0,1,0,0,1,1,...,1,1,1,1,0,1,0,0,0,2


Eliminación de variables no necesarias: Nombre

In [36]:
datos_mod = datos.iloc[:, -25:-1]

In [37]:
datos_mod.head(2)

,edad,sex,rel_ctxo_rel_mala,rel_ctxo_trauma,rel_ctxo_buena,ed_perm,ed_norm,ed_estr,resil_ba,resil_me,...,fil_men,max_min,conc_arb,pseu_res,deb,raz_emo,inhib,asert,agres,impuls
0,20,0,1,0,0,0,1,0,0,1,...,0,1,1,0,1,1,0,0,1,1
1,19,0,0,0,1,0,0,1,1,0,...,0,1,1,1,1,0,1,0,0,0


Centrado y escalado de los datos debido a la variable EDAD

In [38]:
from sklearn.preprocessing import scale

In [39]:
datos_ml = scale(datos_mod)

Ahora que están los datos centrados y escalados y con las variables innecesarias eliminadas, se puede pasar a los métodos de machine learning

In [40]:
# Hay que hacer una separación entre datos de train y test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(datos_ml, datos.iloc[:, 25], random_state=97)

Las redes neuronales han sido desarrolladas en R, así como todo el análisis de datos. A continuación desarrollaré el resto de métodos:

                                                                                        KNN

In [9]:
# Se hace una grid para comprobar cuales serán los mejores parámetros

from sklearn.model_selection import GridSearchCV

grid_knn = {
    'n_neighbors' : range(1, 15),
    'metric' : ['euclidean', 'manhattan']
}

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn = GridSearchCV(KNeighborsClassifier(), grid_knn, cv = 3, scoring='accuracy')

In [11]:
knn.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': range(1, 15), 'metric': ['euclidean', 'manhattan']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [12]:
knn.best_params_

{'metric': 'manhattan', 'n_neighbors': 12}

In [13]:
from sklearn.model_selection import cross_val_score

knn = knn.best_estimator_.fit(x_train, y_train)

puntuacion = knn.score(x_train, y_train)

print("La puntuación máxima conseguida en train es: {:.1f}".format(puntuacion)) 

puntuacion = knn.score(x_test, y_test)

print("La puntuación máxima conseguida en test es: {:.1f}".format(puntuacion)) 

La puntuación máxima conseguida en train es: 0.5
La puntuación máxima conseguida en test es: 0.5


In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
print(confusion_matrix(y_test, knn.predict(x_test)))

[[2 1 0 1]
 [1 7 0 0]
 [0 1 0 0]
 [3 1 0 0]]


                                                                                    Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
randomForest_grid = RandomForestClassifier(random_state=97)

grid_rf = { 
    'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : range(5, 15),
    'criterion' :['gini', 'entropy']
}

#CV por defecto de 3

randomForest = GridSearchCV(estimator=randomForest_grid, param_grid=grid_rf, cv = 3)

In [18]:
randomForest.fit(x_train, y_train)

randomForest.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'n_estimators': 100}

In [19]:
print(randomForest.best_estimator_)
print("")

randomForest = randomForest.best_estimator_.fit(x_train, y_train)

print("Clasificación Random Forest Grid CV 5")

puntuacion = randomForest.score(x_train, y_train)

print("La puntuación en tanto por 1 de train de: {:.3f}".format(puntuacion))

puntuacion = randomForest.score(x_test, y_test)

print("La puntuación en tanto por 1 de test es de: {:.3f}".format(puntuacion))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=97, verbose=0, warm_start=False)

Clasificación Random Forest Grid CV 5
La puntuación en tanto por 1 de train de: 0.920
La puntuación en tanto por 1 de test es de: 0.471


In [20]:
print(confusion_matrix(y_test, randomForest.predict(x_test)))

[[1 2 0 1]
 [2 6 0 0]
 [0 1 0 0]
 [1 2 0 1]]


                                                                                        SVM

In [41]:
from sklearn import svm

In [ ]:
parametros = {
    'kernel':('linear', 'rbf', 'poly'),
    'C':range(1, 10),
    'degree': range(2, 5),
    
}

svc_grid = svm.SVC(gamma="scale", random_state=97)
svc_grid = GridSearchCV(svc_grid, parametros, cv=3)
svc_grid.fit(x_train, y_train)
svc_grid.best_params_

In [ ]:
print("Clasificación SVM Grid CV 3")

svc_grid = svc_grid.best_estimator_.fit(x_train, y_train)

puntuacion = svc_grid.score(x_train, y_train)

print("La puntuación del train en tanto por 1 es de: {:.3f}".format(puntuacion))

puntuacion = svc_grid.score(x_test, y_test)

print("La puntuación del test en tanto por 1 es de: {:.3f}".format(puntuacion))

In [ ]:
print(confusion_matrix(y_test, svc_grid.predict(x_test)))

                                                                                            K-Means

In [24]:
from sklearn.cluster import KMeans

In [25]:
# 4 clusters, uno para cada grupo

kmeans = KMeans(n_clusters=4, random_state=97)

kmeans_model = kmeans.fit(x_train, y_train)

In [26]:
prediction_kmeans = kmeans_model.predict(x_test)

kmeans_model.cluster_centers_

array([[-0.73559281,  0.22393804, -0.10066827,  0.50428397, -0.4149716 ,
         1.58943883, -0.98518437, -0.53708616,  0.67175988, -0.64456732,
        -0.12309149,  0.0146385 ,  0.21650635,  0.12347898,  0.26799143,
         0.1754116 ,  0.12309149, -0.61499388, -0.59214434,  0.51395617,
         0.09116072, -0.3939193 ,  0.18260929, -0.22971146],
       [ 0.87158615,  0.18880023,  0.4439408 ,  0.44469486, -0.72929232,
         0.00473047,  0.30067315, -0.3657301 , -1.14470294,  1.18019369,
        -0.12309149, -0.59251076, -0.47425201, -0.73009398, -0.71586752,
        -0.24432331,  0.12309149, -0.15780009,  0.25197632, -0.18880023,
        -0.02918939,  0.23447577, -0.19437404, -0.08313367],
       [ 0.03983879, -0.07990075, -0.22141869, -0.50172044,  0.63220387,
        -0.62915287,  1.01503844, -0.53708616,  0.39869685, -0.37025684,
        -0.12309149,  0.34156503,  0.21650635,  0.58309519,  0.22458589,
         0.1754116 ,  0.12309149,  0.39688354,  0.00370553,  0.22458589,
  

Para hacer plotting, tengo que reducir a dos dimensiones y entonces mostrarlo:

In [43]:
# Esta representación no es la de KMeans, sino la real sobre las dos dimensiones más significativas

from sklearn.decomposition import PCA

pca = PCA(n_components=2)

comp_p = pca.fit_transform(datos_ml)
df_pca_2 = pd.DataFrame(data = comp_p, columns = ['Componente Principal 1', 'Componente Principal 2'])

df_pca_2 = pd.concat([df_pca_2, datos.iloc[:, 25]], axis = 1)

fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Componente Principal 1', fontsize = 15)
ax.set_ylabel('Componente Principal 2', fontsize = 15)
ax.set_title('2 Componentes Principales', fontsize = 25)
targets = [1, 2, 3, 4]
colors = ['r', 'g', 'b', 'y']

for target, color in zip(targets,colors):
    indicesToKeep = df_pca_2.iloc[:, 2] == target
    ax.scatter(df_pca_2.loc[indicesToKeep, 'Componente Principal 1'], df_pca_2.loc[indicesToKeep, 'Componente Principal 2']
               ,c = color ,s = 50)
    
ax.legend(targets)
plt.savefig("Imágenes Obtenidas/Plot Kmeans.png")
ax.grid()

NameError: name 'plt' is not defined

Se puede observar claramente que es un problema dificilmente clasificable, al menos en dos dimensiones

                                                                            SVM Clases no Balanceadas

In [28]:
parametros = {
    'C':range(1, 5),
    'degree': range(2, 5)
}

svc_grid = svm.SVC(kernel='linear', gamma="scale", random_state=97, class_weight='balanced', probability=True)
svc_grid = GridSearchCV(svc_grid, parametros, cv=3)
svc_grid.fit(x_train, y_train)
svc_grid.best_params_

TypeError: must be real number, not str

In [ ]:
print("Clasificación SVM Grid CV 3 Para Clases No Balanceadas")

svc_grid = svc_grid.best_estimator_.fit(x_train, y_train)

puntuacion = svc_grid.score(x_train, y_train)

print("La puntuación del train en tanto por 1 es de: {:.3f}".format(puntuacion))

puntuacion = svc_grid.score(x_test, y_test)

print("La puntuación del test en tanto por 1 es de: {:.3f}".format(puntuacion))

In [ ]:
print(confusion_matrix(y_test, svc_grid.predict(x_test)))

Técnica Resampling

In [44]:
from sklearn.utils import resample

# Separate majority and minority classes
df_minority1 = datos[datos.iloc[:, 25]==1]
df_majority2 = datos[datos.iloc[:, 25]==2]
df_minority3 = datos[datos.iloc[:, 25]==3]
df_minority4 = datos[datos.iloc[:, 25]==4]
 
# Upsample minority class
df_minority_upsampled1 = resample(df_minority1, 
                                 replace=True,     # Con replacement
                                 n_samples=25,     # Para conseguir esa cantidad
                                 random_state=97)  # Reproducible mediante seed

df_minority_upsampled3 = resample(df_minority3, 
                                 replace=True,     
                                 n_samples=25,    
                                 random_state=97) 

df_minority_upsampled4 = resample(df_minority4, 
                                 replace=True,     
                                 n_samples=25,    
                                 random_state=97) 
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled1, df_majority2, df_minority_upsampled3, df_minority_upsampled4])
 
# Display new class counts
df_upsampled.iloc[:, 25].groupby(df_upsampled.iloc[:, 25]).count()

 grupo
1    25
2    30
3    25
4    25
Name:  grupo, dtype: int64

In [45]:
# Hay que hacer una separación entre datos de train y test
nuevosGrupos = df_upsampled.iloc[:, 25]
datos_mod = df_upsampled.iloc[:, -25:-1]

from sklearn.preprocessing import scale

datos_ml = scale(datos_mod)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(datos_ml, nuevosGrupos, random_state=97)

                                                                                        KNN Balanceado

In [46]:
# Se hace una grid para comprobar cuales serán los mejores parámetros

from sklearn.model_selection import GridSearchCV

grid_knn = {
    'n_neighbors' : range(1, 15),
    'metric' : ['euclidean', 'manhattan']
}

In [47]:
from sklearn.neighbors import KNeighborsClassifier

knn = GridSearchCV(KNeighborsClassifier(), grid_knn, cv = 4, scoring='accuracy')

In [48]:
knn.fit(x_train, y_train)

GridSearchCV(cv=4, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': range(1, 15), 'metric': ['euclidean', 'manhattan']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [49]:
knn.best_params_

{'metric': 'manhattan', 'n_neighbors': 1}

In [50]:
from sklearn.model_selection import cross_val_score

knn = knn.best_estimator_.fit(x_train, y_train)

puntuacion = knn.score(x_train, y_train)

print("La puntuación máxima conseguida en train es: {:.1f}".format(puntuacion)) 

puntuacion = knn.score(x_test, y_test)

print("La puntuación máxima conseguida en test es: {:.1f}".format(puntuacion)) 

La puntuación máxima conseguida en train es: 1.0
La puntuación máxima conseguida en test es: 0.8


In [51]:
from sklearn.metrics import confusion_matrix

In [52]:
print(confusion_matrix(y_test, knn.predict(x_test)))

[[4 0 0 2]
 [0 5 1 2]
 [1 0 8 0]
 [0 0 0 4]]


Como se puede observar, se ha consguido la misma puntuación pero ahora acierta medianamente con todos los grupos

                                                                                Random Forest Balanceado

In [53]:
from sklearn.ensemble import RandomForestClassifier

In [54]:
randomForest_grid = RandomForestClassifier(random_state=97)

grid_rf = { 
    'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : range(5, 15),
    'criterion' :['gini', 'entropy']
}

#CV por defecto de 3

randomForest = GridSearchCV(estimator=randomForest_grid, param_grid=grid_rf, cv = 3)

In [ ]:
randomForest.fit(x_train, y_train)

randomForest.best_params_

In [ ]:
print(randomForest.best_estimator_)
print("")

randomForest = randomForest.best_estimator_.fit(x_train, y_train)

print("Clasificación Random Forest Grid CV 5 Balanceado")

puntuacion = randomForest.score(x_train, y_train)

print("La puntuación en tanto por 1 de train de: {:.3f}".format(puntuacion))

puntuacion = randomForest.score(x_test, y_test)

print("La puntuación en tanto por 1 de test es de: {:.3f}".format(puntuacion))

In [ ]:
print(confusion_matrix(y_test, randomForest.predict(x_test)))

                                                                                            SVM Balanceado

In [ ]:
from sklearn import svm

In [ ]:
parametros = {
    'kernel':('linear', 'rbf', 'poly'),
    'C':range(1, 10),
    'degree': range(2, 5),
    
}

svc_grid = svm.SVC(gamma="scale", random_state=97)
svc_grid = GridSearchCV(svc_grid, parametros, cv=3)
svc_grid.fit(x_train, y_train)
svc_grid.best_params_

In [ ]:
print("Clasificación SVM Grid CV 3 Balanceado")

svc_grid = svc_grid.best_estimator_.fit(x_train, y_train)

puntuacion = svc_grid.score(x_train, y_train)

print("La puntuación del train en tanto por 1 es de: {:.3f}".format(puntuacion))

puntuacion = svc_grid.score(x_test, y_test)

print("La puntuación del test en tanto por 1 es de: {:.3f}".format(puntuacion))

In [ ]:
print(confusion_matrix(y_test, svc_grid.predict(x_test)))

                                                                                        K-Means Balanceado

In [ ]:
from sklearn.cluster import KMeans

# 4 clusters, uno para cada grupo

kmeans = KMeans(n_clusters=4, random_state=97)

kmeans_model = kmeans.fit(x_train, y_train)
prediction_kmeans = kmeans_model.predict(x_test)

kmeans_model.cluster_centers_


In [ ]:
# Esta representación no es la de KMeans, sino la real sobre las dos dimensiones más significativas

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)

comp_p = pca.fit_transform(datos_ml)
df_pca_2 = pd.DataFrame(data = comp_p, columns = ['Componente Principal 1', 'Componente Principal 2'])

df_pca_2 = pd.concat([df_pca_2, datos.iloc[:, 25]], axis = 1)

fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Componente Principal 1', fontsize = 15)
ax.set_ylabel('Componente Principal 2', fontsize = 15)
ax.set_title('2 Componentes Principales', fontsize = 25)
targets = [1, 2, 3, 4]
colors = ['r', 'g', 'b', 'y']

for target, color in zip(targets,colors):
    indicesToKeep = df_pca_2.iloc[:, 2] == target
    ax.scatter(df_pca_2.loc[indicesToKeep, 'Componente Principal 1'], df_pca_2.loc[indicesToKeep, 'Componente Principal 2']
               ,c = color ,s = 50)
    
ax.legend(targets)
plt.savefig("Imágenes Obtenidas/Plot Kmeans Resampled.png")
ax.grid()

                                                                                                                            Deep Learning

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Inicialización del constructor
modelo = Sequential()

# Add an input layer 
modelo.add(Dense(24, activation='relu', input_shape=(24,)))

# Add one hidden layer 
modelo.add(Dense(12, activation='relu'))

# Add an output layer 
modelo.add(Dense(4, activation='relu'))

In [ ]:
# Dimensiones de output del modelo
# modelo.output_shape

# Resumen del modelo
modelo.summary()

# Configuración del modelo
#modelo.get_config()

# Lista de los pesos
#modelo.get_weights()

In [ ]:
modelo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
modelo.fit(x_train, y_train,epochs=25, batch_size=1, verbose=1)

In [ ]:
y_pred = modelo.predict(x_test)
y_pred

In [ ]:
score = modelo.evaluate(x_train, y_train,verbose=1)

print(score)

In [ ]:
score = modelo.evaluate(x_test, y_test,verbose=1)

print(score)

In [ ]:
import numpy as np

confusion_matrix(y_test,np.round(y_pred))